<a href="https://colab.research.google.com/github/fjadidi2001/Cyber-Attack-Detection/blob/main/satellite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cell 1: Setup and Dataset Download



In [3]:
# Install required libraries
!pip install earthengine-api geemap rasterio scikit-image pandas matplotlib -q

# Import libraries
import ee
import geemap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu
import rasterio
from rasterio.plot import show
import os

# Authenticate Google Earth Engine
try:
    ee.Authenticate()  # Run this and follow the link to authenticate
except Exception as e:
    print(f"Authentication error: {e}")

# Initialize Earth Engine with your Google Cloud Project ID
# Replace 'your-project-id' with your actual Google Cloud Project ID
project_id = 'your-project-id'  # CHANGE THIS TO YOUR PROJECT ID
try:
    ee.Initialize(project=project_id)
    print("Earth Engine initialized successfully!")
except Exception as e:
    print(f"Error initializing Earth Engine: {e}")
    print("Ensure you have a valid Google Cloud Project ID and have authenticated correctly.")

# Define Region of Interest (ROI) - Amazon rainforest example
roi = ee.Geometry.Rectangle([-60.5, -3.5, -60.0, -3.0])  # [minLon, minLat, maxLon, maxLat]

# Output: Table confirming setup
setup_data = {
    'Component': ['Libraries Installed', 'Earth Engine Authenticated', 'Earth Engine Initialized', 'ROI Defined'],
    'Status': ['Completed', 'Completed', 'Completed', f'Rectangle: {roi.getInfo()["coordinates"]}']
}
setup_df = pd.DataFrame(setup_data)
display(setup_df)

# Output: Map of ROI
m = geemap.Map()
m.centerObject(roi, zoom=10)
m.addLayer(roi, {}, 'Region of Interest')
m

Error initializing Earth Engine: Caller does not have required permission to use project your-project-id. Grant the caller the roles/serviceusage.serviceUsageConsumer role, or a custom role with the serviceusage.services.use permission, by visiting https://console.developers.google.com/iam-admin/iam/project?project=your-project-id and then retry. Propagation of the new permission may take a few minutes.
Ensure you have a valid Google Cloud Project ID and have authenticated correctly.


EEException: Caller does not have required permission to use project your-project-id. Grant the caller the roles/serviceusage.serviceUsageConsumer role, or a custom role with the serviceusage.services.use permission, by visiting https://console.developers.google.com/iam-admin/iam/project?project=your-project-id and then retry. Propagation of the new permission may take a few minutes.